In [ ]:
# ! pip install vanna[chromadb,openai] -q

In [1]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore


c:\Users\0200704\.conda\envs\tf-gpu\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (2022server). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [2]:
openai_api = ' '

In [3]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': openai_api, 'model': 'gpt-4o'})

In [4]:
vn.connect_to_sqlite('QRM.db')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [5]:
# New way to train the Question and SQL pair 
# 
vn.train(
    question="How many months of data we have in Utility?", 
    sql="SELECT COUNT(DISTINCT strftime('%Y-%m', date_column)) AS months FROM Utility"
)

'832a69b8-3cbf-5068-8ec4-dc1f2fd7c19f-sql'

In [7]:
question = "How many months of data we have in Utility?"
vn.generate_sql(question)

# Above function will call following functions

# get_similar_question_sql
vn.get_similar_question_sql(question)

# get_related_ddl
vn.get_related_ddl(question) 
# get_related_documentation
vn.get_related_documentation(question)
# get_sql_prompt
vn.get_sql_prompt(
    initial_prompt = None,
    question="How many months of data do we have in Utility?",
    question_sql_list=[
        {
            "question": "How many months of data do we have in Utility?",
            "sql": "SELECT COUNT(DISTINCT strftime('%Y-%m', date_column)) AS months FROM utility"
        }
    ],
    ddl_list=[
        "CREATE TABLE utility (id INT, date_column DATE, usage DECIMAL, cost DECIMAL)"
    ],
    doc_list=[
        "The utility table contains utility usage data, including date, usage amount, and cost."
    ],
)
# submit_prompt

Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "utility" (\n"Month-Year" TEXT,\n  "Product" TEXT,\n  "Utility" TEXT,\n  "Particulars" TEXT,\n  "UOM" TEXT,\n  "Bplan" REAL,\n  "Target" REAL,\n  "Actual" REAL,\n  "Bplan  Var %" TEXT,\n  "Target  Var %" TEXT,\n  "Plant" TEXT\n)\n\n\n===Additional Context \n\nThe \'Utility\' table contains the Utility for QRM data.\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n

Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


LLM Response: SELECT COUNT(DISTINCT "Month-Year") AS number_of_months FROM utility;
Extracted SQL: SELECT COUNT(DISTINCT "Month-Year") AS number_of_months FROM utility;


Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


[{'role': 'system',
  'content': "You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE utility (id INT, date_column DATE, usage DECIMAL, cost DECIMAL)\n\n\n===Additional Context \n\nThe utility table contains utility usage data, including date, usage amount, and cost.\n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, please explain why it can't be generated. \n4. Please use the most relevant table(s). \n5. If the que

In [12]:
# For run the generated sql query

df = vn.run_sql("SELECT * FROM utility")

In [13]:
# Will generate the plotly code
# Need R&D
vn.generate_plotly_code()

Using model gpt-4o for 98.25 tokens (approx)


'import plotly.express as px\nimport plotly.graph_objects as go\n\nif df.size == 1:\n    fig = go.Figure(go.Indicator(\n        mode = "number",\n        value = df.iloc[0, 0]\n    ))\nelse:\n    fig = px.line(df)\n    \n'

In [14]:
# Aboves code output is input for this function


fig = vn.get_plotly_figure(
    plotly_code="fig = px.bar(df, x='name', y='salary')",
    df=df
)
fig.show()